In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-j1ac6dd2/unsloth_43fb50d3468e41a0acdd19bfa28d1d80
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-j1ac6dd2/unsloth_43fb50d3468e41a0acdd19bfa28d1d80
  Resolved https://github.com/unslothai/unsloth.git to commit e6d0877695452b1291ffa17f961397a3a4c21941
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.5/166.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:0

In [2]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Qwen2-0.5B-bnb-4bit"

dataset_name = "yahma/alpaca-cleaned"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.9: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [3]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}"""
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset(dataset_name, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md: 0.00B [00:00, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",]
)

from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth 2025.7.9 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


Unsloth: Tokenizing ["text"]:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [5]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 8,798,208 of 502,830,976 (1.75% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fazil-shaikh24 (fazil-shaikh24-sardar-patel-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,1.779500
2,2.164900
3,1.952100
4,2.115600
5,1.819900
6,1.801600
7,1.303500
8,1.576200
9,1.438800
10,1.469600


Unsloth: Will smartly offload gradients to save VRAM!


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=60, training_loss=1.3501457124948502, metrics={'train_runtime': 293.1343, 'train_samples_per_second': 1.637, 'train_steps_per_second': 0.205, 'total_flos': 278450117199360.0, 'train_loss': 1.3501457124948502})

In [6]:
model.save_pretrained("qwen2_0.5b_finetuned")
tokenizer.save_pretrained("qwen2_0.5b_finetuned")

('qwen2_0.5b_finetuned/tokenizer_config.json',
 'qwen2_0.5b_finetuned/special_tokens_map.json',
 'qwen2_0.5b_finetuned/chat_template.jinja',
 'qwen2_0.5b_finetuned/vocab.json',
 'qwen2_0.5b_finetuned/merges.txt',
 'qwen2_0.5b_finetuned/added_tokens.json',
 'qwen2_0.5b_finetuned/tokenizer.json')

In [7]:
from unsloth import FastLanguageModel
import torch

saved_model_path = "qwen2_0.5b_finetuned"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = saved_model_path,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.7.9: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896, padding_idx=151646)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Line

In [9]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What are the main Reason for adopting LLMs in industry

### Input:
LLMs are becoming increasingly popular in industry.

### Response:
"""

inputs = tokenizer(
[
    prompt
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
response = tokenizer.batch_decode(outputs)

print(response[0])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What are the main Reason for adopting LLMs in industry

### Input:
LLMs are becoming increasingly popular in industry.

### Response:
There are several reasons why companies are adopting LLMs in their industry:

1. Cost Savings: LLMs are becoming increasingly affordable and cost-effective, making them a more cost-effective option for companies to adopt.

2. Increased Efficiency: LLMs can help companies streamline their processes, reduce errors, and increase productivity.

3. Improved Customer Experience: LLMs can provide a more personalized and engaging experience for customers, making them more likely to stay engaged and repeat business.

4. Increased Revenue: LLMs can help companies increase their revenue by automating tasks, reducing costs, and improving customer experience.

5. Better Communication:


#DPO

In [10]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

In [26]:
model_name = "unsloth/Qwen2-0.5B-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

==((====))==  Unsloth 2025.7.9: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [27]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

In [28]:
sft_math_dataset_name = "meta-math/MetaMathQA"
sft_math_dataset = load_dataset(sft_math_dataset_name, split="train")

In [29]:
math_prompt_template = """Below is a mathematical question. Your task is to provide a detailed, step-by-step solution that accurately answers the question.

### Question:
{}

### Solution:
{}"""

def format_math_sft(examples):
    questions = examples["query"]
    solutions = examples["response"]
    texts = []
    for question, solution in zip(questions, solutions):
        text = math_prompt_template.format(question, solution + tokenizer.eos_token)
        texts.append(text)
    return {"text": texts}

sft_math_dataset = sft_math_dataset.map(format_math_sft, batched=True)

Map:   0%|          | 0/395000 [00:00<?, ? examples/s]

In [30]:
sft_trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=sft_math_dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="sft_math_outputs",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/395000 [00:00<?, ? examples/s]

In [31]:
sft_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 395,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 8,798,208 of 502,830,976 (1.75% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
1,0.663800,0,0,0,0,0,0,0,0,0,0,0,0
2,0.713600,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log
3,0.696800,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log
4,0.665800,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log
5,0.655800,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log
6,0.549400,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log
7,0.624500,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log
8,0.546100,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log
9,0.541900,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log
10,0.444900,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log,No Log


TrainOutput(global_step=100, training_loss=0.3384859839081764, metrics={'train_runtime': 187.6541, 'train_samples_per_second': 4.263, 'train_steps_per_second': 0.533, 'total_flos': 578431175523840.0, 'train_loss': 0.3384859839081764})

In [38]:
from trl import DPOTrainer, DPOConfig

dpo_math_dataset_name = "Intel/orca_dpo_pairs"
dpo_math_dataset = load_dataset(dpo_math_dataset_name, split="train")

In [39]:
def format_dpo_math(example):
    if example.get("system"):
        prompt = f"{example['system']}\n{example['question']}"
    else:
        prompt = example['question']
    return {"prompt": prompt, "chosen": example['chosen'], "rejected": example['rejected']}

dpo_math_dataset = dpo_math_dataset.map(format_dpo_math, remove_columns=['system', 'question'])

Map:   0%|          | 0/12859 [00:00<?, ? examples/s]

In [42]:
dpo_config = DPOConfig(
    beta=0.1,

    output_dir="dpo_math_outputs",
    max_steps=100,
    #num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    warmup_ratio=0.1,
    logging_steps=1,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    optim="adamw_8bit",
    seed=3407,
    remove_unused_columns=False,
)

dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=dpo_config,
    train_dataset=dpo_math_dataset,
    tokenizer=tokenizer,
)

In [43]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,859 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 3,862,528 of 502,830,976 (0.77% trained)
/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:928: UserWarning: The model contains some adapter layers that are enabled and others that are disabled. This is most likely unintentional. After exiting the disable_adapter context, all adapters will be enabled
  warnings.warn(


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
1,1.010200,0.087651,0.582465,0.250000,-0.494813,-144.339661,-336.825256,-3.200406,-3.205979,0,0,0,0
2,1.244000,-0.308696,0.477680,0.250000,-0.786375,-208.793015,-319.814514,-2.905146,-3.058169,No Log,No Log,No Log,No Log
3,0.869200,-0.131859,0.156213,0.500000,-0.288073,-257.618286,-208.809082,-3.102028,-3.200707,No Log,No Log,No Log,No Log
4,0.933200,0.086877,0.463205,0.250000,-0.376327,-209.294037,-170.651154,-2.717808,-3.191015,No Log,No Log,No Log,No Log
5,0.920400,0.213781,0.588322,0.250000,-0.374541,-407.782227,-305.274658,-3.157687,-3.365060,No Log,No Log,No Log,No Log
6,0.535000,0.376814,-0.068685,0.750000,0.445499,-145.870605,-254.146484,-3.095010,-3.105019,No Log,No Log,No Log,No Log
7,0.722800,0.183556,0.206085,0.500000,-0.022530,-316.456665,-372.579224,-3.037471,-3.083263,No Log,No Log,No Log,No Log
8,0.851400,-0.104929,0.106287,0.250000,-0.211216,-74.265030,-143.005829,-2.933227,-3.201818,No Log,No Log,No Log,No Log
9,0.755200,0.222716,0.312353,0.500000,-0.089637,-386.807861,-357.947540,-2.873778,-3.209268,No Log,No Log,No Log,No Log
10,0.581300,0.155266,-0.115596,0.500000,0.270861,-171.345642,-228.806885,-2.938114,-3.185478,No Log,No Log,No Log,No Log


TrainOutput(global_step=100, training_loss=0.739167018532753, metrics={'train_runtime': 281.3237, 'train_samples_per_second': 1.422, 'train_steps_per_second': 0.355, 'total_flos': 0.0, 'train_loss': 0.739167018532753, 'epoch': 0.03110661793296524})

In [51]:
model.save_pretrained("qwen2_0.5b_math_finetuned")
tokenizer.save_pretrained("qwen2_0.5b_math_finetuned")

('qwen2_0.5b_math_finetuned/tokenizer_config.json',
 'qwen2_0.5b_math_finetuned/special_tokens_map.json',
 'qwen2_0.5b_math_finetuned/chat_template.jinja',
 'qwen2_0.5b_math_finetuned/vocab.json',
 'qwen2_0.5b_math_finetuned/merges.txt',
 'qwen2_0.5b_math_finetuned/added_tokens.json',
 'qwen2_0.5b_math_finetuned/tokenizer.json')

In [50]:
FastLanguageModel.for_inference(model)

test_question = "A car dealership has 100 cars. 40% of the cars are red, and the rest are blue. If they sell 25% of the red cars, how many red cars are sold and how many red are remaining?"

inputs = tokenizer(
[
    math_prompt_template.format(test_question, "")
], return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)
response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print("--- Question ---")
print(test_question)
print("\n--- Generated Solution ---")
solution_start = response_text.find("### Solution:")
print(response_text[solution_start:])

--- Question ---
A car dealership has 100 cars. 40% of the cars are red, and the rest are blue. If they sell 25% of the red cars, how many red cars are sold and how many red are remaining?

--- Generated Solution ---
### Solution:
The dealership has 100 * 0.4 = 40 red cars.
They sell 40 * 0.25 = 10 red cars.
There are 40 - 10 = 30 red cars remaining.
#### 30
The answer is: 30
